In [244]:
values = [1,2,3,4,5,6]
base_configs = [[2,4,5,3],
                [6,4,1,3],
                [2,1,5,6],
                [2,6,5,1],
                [1,4,6,3],
                [5,4,2,3]]

base_configs = (np.array(base_configs) - 1).astype(list)

print(base_configs)

direction_dict = {
    'U' : 2,
    'D' : 0,
    'L' : 1,
    'R' : 3
}

opposite_direction_dict = {
    'U' : 'D',
    'D' : 'U',
    'L' : 'R',
    'R' : 'L'
}

ninety_clock_dict = {
    'U' : 'R',
    'D' : 'L',
    'L' : 'U',
    'R' : 'D'
}

ninety_anticlock_dict = {
    'U' : 'L',
    'D' : 'R',
    'L' : 'D',
    'R' : 'U'
}

direction_vector_dict = {
    'U' : [-1,0],
    'D' : [1,0],
    'L' : [0,-1],
    'R' : [0,1]
}

class Face:
    def __init__(self,number,configs):
        self.configs = configs
        self.number = number
        self.current_config = configs[0]
    def turn(self,dir):
        new_top_face = self.current_config[direction_dict[dir]]
        # ! A,B,C
        # ! New Top Number, B will be the Cth character in the config
        return new_top_face,self.number,direction_dict[opposite_direction_dict[dir]]
    def reset_config(self,num,idx):
        #print(self.configs)
        for config in self.configs:
            if config[idx] == num:
                self.current_config = config
    
class Dice:
    def __init__(self):
        self.faces = []
        for x in range(6):
            config = list(base_configs[x])
            possible_configs = []
            for i in range(4):
                possible_configs.append(config[:])
                saved_value = config.pop(0)
                config.append(saved_value)
            self.faces.append(Face(x+1,possible_configs))
        self.current_face = 0
        self.current_config = None
        self.heading = 'U'
    def rotate(self,dir):
        new_top_face,number,index = self.faces[self.current_face].turn(dir)
        #print("GRUSELIG",number-1,index+1)
        self.current_face = new_top_face
        self.faces[new_top_face].reset_config(number-1,index)
        self.current_config = self.faces[new_top_face].current_config
        self.heading = dir

[[1 3 4 2]
 [5 3 0 2]
 [1 0 4 5]
 [1 5 4 0]
 [0 3 5 2]
 [4 3 1 2]]


In [245]:
dice = Dice()

In [246]:
import numpy as np
class Grid:
    def __init__(self):
        self.values = np.ones((11,11))
        self.pos = np.array([5,5])
        self.dice = Dice()
    def update(self):
        #print(self.dice.current_face)
        #print(self.dice.current_config)
        print(self.pos)
        value = self.values[self.pos[0]][self.pos[1]]+self.dice.current_face+1
        print(value)
        if value > 6:
            value -= 6
        self.values[self.pos[0]][self.pos[1]] = value
        heading = self.dice.heading
        print(value)
        if value == 1 or value == 6:
            direction = heading
        elif value == 2:
            print("YEAH")
            direction = ninety_clock_dict[heading]
        elif value == 3 or value == 4:
            direction = opposite_direction_dict[heading]
        elif value == 5:
            direction = ninety_anticlock_dict[heading]
        self.dice.rotate(direction)
        print(direction,self.dice.current_face+1)
        if (self.pos + direction_vector_dict[direction])[0] > 10:
            self.pos += direction_vector_dict[direction]
            self.pos[0] = 0
        elif (self.pos + direction_vector_dict[direction])[0] < 0:
            self.pos += direction_vector_dict[direction]
            self.pos[0] = 10
        elif (self.pos + direction_vector_dict[direction])[1] < 0:
            self.pos += direction_vector_dict[direction]
            self.pos[1] = 10
        elif (self.pos + direction_vector_dict[direction])[1] > 10:
            self.pos += direction_vector_dict[direction]
            self.pos[1] = 0
        else:
            self.pos += direction_vector_dict[direction]
    def set_grid(self,array):
        base = np.array([4,4])
        for x in range(3):
            for y in range(3):
                new_base = base +[x,y]
                self.values[new_base[0]][new_base[1]] = array[x][y]
    def print_grid(self):
        base = self.pos-[1,1]
        print(base)
        for i in range(3):
            #print(base[0]+i,base[1]+i)
            #print(base[0]+i,base[1]+i+1)
            #print(base[0]+i,base[1]+i+2)
            if base[0]+i > 10 or base[0]+i < 0 or base[1] > 10 or base[1] < 0:
                value_1 = 'X'
            else:
                value_1 = int(self.values[base[0]+i][base[1]])
            
            if base[0]+i > 10 or base[0]+i < 0 or base[1]+1 > 10 or base[1]+1 < 0:
                value_2 = 'X'
            else:
                value_2 = int(self.values[base[0]+i][base[1]+1])
                
            if base[0]+i > 10 or base[0]+i < 0 or base[1]+2 > 10 or base[1]+2 < 0:
                value_3 = 'X'
            else:
                value_3 = int(self.values[base[0]+i][base[1]+2])
            print(value_1,
                  value_2,
                  value_3)

In [248]:
threebythree = [[1,3,5],
                [1,6,5],
                [1,1,5]]
grid = Grid()
grid.set_grid(threebythree)
moves = 1
for i in range(5):
    grid.update()
    print(grid.values)
grid.print_grid()

[5 5]
7.0
1.0
U 5
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 3. 5. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 5. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 5. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[4 5]
8.0
2.0
YEAH
R 3
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 2. 5. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 5. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 5. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[4 6]
8.0
2.0
YEAH
D 1
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [215]:
moves = 1
for i in range(1):
    grid.update()
grid.print_grid()

[4 5]
3.0 3
R 5
[4 6]
5 1 1
5 1 1
5 1 1


In [121]:
grid = Grid()

0
None
GRUSELIG 0 3
[[5, 3, 0, 2], [3, 0, 2, 5], [0, 2, 5, 3], [2, 5, 3, 0]]
U 1 [5 4]

1
[5, 3, 0, 2]
GRUSELIG 1 4
[[1, 5, 4, 0], [5, 4, 0, 1], [4, 0, 1, 5], [0, 1, 5, 4]]
R 3 [5 5]

3
[5, 4, 0, 1]
GRUSELIG 3 2
[[5, 3, 0, 2], [3, 0, 2, 5], [0, 2, 5, 3], [2, 5, 3, 0]]
L 1 [4 5]

1
[5, 3, 0, 2]
GRUSELIG 1 3
[[4, 3, 1, 2], [3, 1, 2, 4], [1, 2, 4, 3], [2, 4, 3, 1]]
U 5 [4 4]

5
[4, 3, 1, 2]
GRUSELIG 5 3
[[0, 3, 5, 2], [3, 5, 2, 0], [5, 2, 0, 3], [2, 0, 3, 5]]
U 4 [4 3]

4
[0, 3, 5, 2]
GRUSELIG 4 2
[[1, 0, 4, 5], [0, 4, 5, 1], [4, 5, 1, 0], [5, 1, 0, 4]]
L 2 [3 3]

2
[0, 4, 5, 1]
GRUSELIG 2 4
[[0, 3, 5, 2], [3, 5, 2, 0], [5, 2, 0, 3], [2, 0, 3, 5]]
R 4 [3 4]

4
[0, 3, 5, 2]
GRUSELIG 4 3
[[1, 3, 4, 2], [3, 4, 2, 1], [4, 2, 1, 3], [2, 1, 3, 4]]
U 0 [3 3]

0
[1, 3, 4, 2]
GRUSELIG 0 3
[[5, 3, 0, 2], [3, 0, 2, 5], [0, 2, 5, 3], [2, 5, 3, 0]]
U 1 [3 2]

1
[5, 3, 0, 2]
GRUSELIG 1 4
[[1, 5, 4, 0], [5, 4, 0, 1], [4, 0, 1, 5], [0, 1, 5, 4]]
R 3 [3 3]



In [77]:
grid.pos

array([5, 4])

In [44]:
dice = Dice()
dice.rotate("U")

In [45]:
dice.current_face

1

In [21]:
faces[1].number

2